<a href="https://colab.research.google.com/github/SanghunOh/test_deeplearning/blob/main/iris_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn import datasets

In [ ]:
iris = datasets.load_iris()
iris

In [8]:
import pandas as pd
df_iris = pd.DataFrame(iris.data)
df_iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       150 non-null    float64
 1   1       150 non-null    float64
 2   2       150 non-null    float64
 3   3       150 non-null    float64
dtypes: float64(4)
memory usage: 4.8 KB


In [9]:
import sqlite3
connect = sqlite3.connect('./db.sqlite3')
df_iris.to_sql('iris_resource', connect, if_exists='append', index=False)

In [ ]:
df_load = pd.read_sql_query('select * from iris_resource', connect)
df_load.head(4)

In [11]:
x_data = df_load.to_numpy()
x_data.shape

(150, 4)

In [15]:
import numpy as np

In [16]:
y_data = iris.target
y_data, np.unique(y_data)

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]),
 array([0, 1, 2]))

In [13]:
import tensorflow as tf

In [20]:
model = tf.keras.Sequential()

model.add(tf.keras.Input(shape=(4,))) # input layer

model.add(tf.keras.layers.Dense(64, activation='relu')) # hidden layer
model.add(tf.keras.layers.Dense(24, activation='relu')) # hidden layer

model.add(tf.keras.layers.Dense(3, activation='softmax')) # output layer


model.compile(optimizer='adam', loss='mse', metrics=['acc'], ) # gadget

In [ ]:
model.summary()

In [21]:
model.fit(x_data, y_data, epochs=50, validation_split=0.3) # 학습

Epoch 1/50
4/4 [==============================] - 1s 48ms/step - loss: 0.4068 - acc: 0.4762 - val_loss: 2.7883 - val_acc: 0.5778
Epoch 2/50
4/4 [==============================] - 0s 13ms/step - loss: 0.4002 - acc: 0.3619 - val_loss: 2.7834 - val_acc: 1.0000
Epoch 3/50
4/4 [==============================] - 0s 7ms/step - loss: 0.3989 - acc: 0.5238 - val_loss: 2.7800 - val_acc: 1.0000
Epoch 4/50
4/4 [==============================] - 0s 7ms/step - loss: 0.3991 - acc: 0.5238 - val_loss: 2.7793 - val_acc: 1.0000
Epoch 5/50
4/4 [==============================] - 0s 7ms/step - loss: 0.3987 - acc: 0.5238 - val_loss: 2.7790 - val_acc: 1.0000
Epoch 6/50
4/4 [==============================] - 0s 7ms/step - loss: 0.3980 - acc: 0.5238 - val_loss: 2.7791 - val_acc: 0.8444
Epoch 7/50
4/4 [==============================] - 0s 9ms/step - loss: 0.3975 - acc: 0.6190 - val_loss: 2.7790 - val_acc: 0.3778
Epoch 8/50
4/4 [==============================] - 0s 8ms/step - loss: 0.3974 - acc: 0.7714 - val_loss:

# Evaluation

In [22]:
model.evaluate(x_data, y_data)

5/5 [==============================] - 0s 2ms/step - loss: 1.1112 - acc: 0.2133


[1.1111743450164795, 0.2133333384990692]

# service

In [24]:
x_data[25], y_data[25]

(array([5. , 3. , 1.6, 0.2]), 0)

In [27]:
pred = model.predict([[5. , 3. , 1.6, 0.2]])
pred

array([[0.33113125, 0.33784267, 0.33102605]], dtype=float32)

In [28]:
import numpy as np
np.argmax(pred)

1